In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_mistralai import ChatMistralAI
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_API_KEY")
os.environ['NVIDIA_API_KEY']=os.getenv("NVIDIA_API_KEY")

In [2]:
model=ChatMistralAI()

In [3]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

/var/folders/s7/sr0hn1c14qg6y6gmb6f2dg3c0000gn/T/ipykernel_48500/1249186930.py:5: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceBgeEmbeddings(
/Users/sidharthjain/Desktop/Data Science Projects/Chatbot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(hf)

In [5]:
file_path="/Users/sidharthjain/Desktop/Data Science Projects/Chatbot/RAG chatbot/BTP_Final_Report (1).pdf"

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
docs=loader.load()

In [7]:
(docs[0].page_content)

'UNVEILINGTHEPOWEROFRANDOMSUBSPACESELECTION:ACOMPREHENSIVEEVALUATION\nAREPORTSUBMITTEDINPARTIALFULFILLMENTOFTHEREQUIREMENTSFORTHEAWARDOFTHEDEGREEOFBACHELOROFTECHNOLOGYINDEPARTMENTOFINFORMATIONTECHNOLOGY\nSubmittedby:AbhishekBansal(2020UIN3309)AakritiGupta(2020UIN3317)SidharthJain(2020UIN3360)\nUnderthesupervisionofDr.AnkitaBansal\nDEPARTMENTOFINFORMATIONTECHNOLOGYNETAJISUBHASUNIVERSITYOFTECHNOLOGYMay,2024\ni'

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)



In [9]:
(all_splits[0].page_content)

'UNVEILINGTHEPOWEROFRANDOMSUBSPACESELECTION:ACOMPREHENSIVEEVALUATION\nAREPORTSUBMITTEDINPARTIALFULFILLMENTOFTHEREQUIREMENTSFORTHEAWARDOFTHEDEGREEOFBACHELOROFTECHNOLOGYINDEPARTMENTOFINFORMATIONTECHNOLOGY\nSubmittedby:AbhishekBansal(2020UIN3309)AakritiGupta(2020UIN3317)SidharthJain(2020UIN3360)\nUnderthesupervisionofDr.AnkitaBansal\nDEPARTMENTOFINFORMATIONTECHNOLOGYNETAJISUBHASUNIVERSITYOFTECHNOLOGYMay,2024\ni'

In [10]:
vector_1 = hf.embed_query(all_splits[0].page_content)

/Users/sidharthjain/Desktop/Data Science Projects/Chatbot/venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [13]:
len(vector_1)

384

In [ ]:
document_ids = vector_store.add_documents(documents=all_splits)

In [ ]:
len(document_ids)

In [58]:
retrieved_result=vector_store.similarity_search("Sidharth Jain")

In [59]:
print(retrieved_result[0].page_content)

UNVEILINGTHEPOWEROFRANDOMSUBSPACESELECTION:ACOMPREHENSIVEEVALUATION
AREPORTSUBMITTEDINPARTIALFULFILLMENTOFTHEREQUIREMENTSFORTHEAWARDOFTHEDEGREEOFBACHELOROFTECHNOLOGYINDEPARTMENTOFINFORMATIONTECHNOLOGY
Submittedby:AbhishekBansal(2020UIN3309)AakritiGupta(2020UIN3317)SidharthJain(2020UIN3360)
UnderthesupervisionofDr.AnkitaBansal
DEPARTMENTOFINFORMATIONTECHNOLOGYNETAJISUBHASUNIVERSITYOFTECHNOLOGYMay,2024
i
